In [1]:
from __future__ import print_function, division
import shutil, sys  
import torch
import torch.nn as nn
import Reconstruction_dataset as dt
import model_4_o as mdo
import model_ResUGAN as md
import numpy as np
import os, glob
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
import matplotlib.pyplot as plt
from torchvision import transforms, datasets, utils, models
import pandas as pd
import math
import torchvision
from skimage import io, transform, img_as_float, exposure
import cv2
from tkinter.filedialog import askopenfilename, askopenfilenames
from skimage.transform import rescale, resize, downscale_local_mean
import imagej
ij=imagej.init(r'C:\fiji-win64\Fiji.app')

Added 375 JARs to the Java classpath.


In [2]:
def get_image_names():
    
    filenames = askopenfilenames()
    
    return filenames

In [3]:
def image_padding(image, stride, patch_size):
    
    y_size = image.shape[0]
    x_size = image.shape[1]
    (dy,my)=divmod(y_size-patch_size,stride)
    (dx,mx)=divmod(x_size-patch_size,stride)
    dif_y=stride-my
    dif_x=stride-mx
    if (my > 0 ):
        
        new_y_size = y_size + dif_y
        pad_y = dif_y
        
    else:
    
        new_y_size = y_size
        pad_y = my
    if (mx > 0 ):   
        new_x_size = x_size + dif_x
        pad_x = dif_x
    else:
        new_x_size = x_size
        pad_x = mx
    
    image_resized = resize(image, (new_y_size, new_x_size))
    padded_image= cv2.copyMakeBorder(image,0,pad_y,0,pad_x,cv2.BORDER_CONSTANT,value=0)
    
    return (padded_image,pad_y,pad_x)

In [4]:
def patch_num_calculator(image,patch_size, stride):
    
    y_size = image.shape[0]
    x_size = image.shape[1]

    rows_patch_num = int(math.floor((image.shape[0]-patch_size)/stride)+1)
    culs_patch_num = int(math.floor((image.shape[1]-patch_size)/stride)+1)

    return (rows_patch_num,culs_patch_num)

In [15]:
def image_to_minibatch(image, stride,patch_size, rows_patch_num,culs_patch_num, unpadd_patch, HE_name):
    
    path = os.getcwd()  
           
    output_path1 =os.path.join(path, "Patches", "SA_Stroma_SHG")
    img_name = HE_name.split('/')[-1].split('.tif')[0]
    output_path =os.path.join(output_path1, img_name)

    if ( os.path.exists(output_path) != True):
            os.mkdir(output_path) 
                                  
    config_file=os.path.join(output_path, "TileConfiguration.txt")
    config = open(config_file,"w+")
    openning_text = "# Define the number of dimensions we are working on" + "\n" + "dim = 2" + "\n\n" + "# Define the image coordinates"+ "\n"
    config.write(openning_text)
    to_be_cropped_row=[]
    to_be_cropped_col=[]
    for i in range(rows_patch_num):
        for j in range(culs_patch_num):

             HEpatch=image[i*stride : i*stride + patch_size, j*stride : j*stride + patch_size]
             col = i*stride - unpadd_patch
             row = j*stride - unpadd_patch
             image_name = img_name + "_SHGpatch_" + i.__str__() +"_" + j.__str__() +".tif"
             image_path=os.path.join(output_path, image_name)
             coordinates = "; ;" + " (" + row.__str__()  + "," + col.__str__()  + ")" + "\n"
             config_text = image_name + coordinates
             config.write(config_text)
             io.imsave(image_path,HEpatch)
             if (i == rows_patch_num-1):
                to_be_cropped_row.append(image_name)
             if (j == culs_patch_num-1):
                to_be_cropped_col.append(image_name)
    config.close()
    return (output_path, to_be_cropped_row, to_be_cropped_col)

In [16]:
unpadd_patch=0
stride=96
patch_size= 128
minibatch_size = 128
HE_names = get_image_names()
for i in range(HE_names.__len__()):
    HE_name = ''.join(HE_names[i])
    HE_image=io.imread(HE_name)
    (padded_image,pad_y,pad_x)=image_padding(HE_image, stride, patch_size)

    (rows_patch_num,culs_patch_num) = patch_num_calculator(padded_image,patch_size, stride)
    (output_path, to_be_cropped_row, to_be_cropped_col) = image_to_minibatch(padded_image, 
                                                                                          stride,patch_size, 
                                                                                          rows_patch_num,
                                                                                          culs_patch_num,unpadd_patch,
                                                                                          HE_name)

In [ ]:
HE_name